In [1]:
import gspread
import datetime

In [2]:
from google.oauth2.service_account import Credentials

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    'broadstreet-gk.json',
    scopes=scopes
)

gc = gspread.authorize(credentials)

In [7]:
#open the google sheet, the string could eventually be in a var, populated by user? 
#sh = gc.open('Copy of West-Prairie-RiverValley COVID-19 by County (TEAM ENTRY) 12-02-2020 to present')



## Create a function to choose Google Sheet based on user input
Inputs:
State - String (or FIP? - int)
Date - String?
County - String (optional)


d1 = "2020-11-29"
d2 = "2020-11-29"
print(d1<=d2)
True

**MAKE SURE TO SHARE ALL SHEETS WITH API EMAIL**


*executing sheetFinder will execute leapyear and regionFinder*

## What does sheetFinder do?
Establishes appropriate connection.

- if date+7 doesnt exceed upper bound:

Returns dictionary of sheet connection information (the name of the sheet)
- if date+7 exceeds upper bound:

Returns dictionary of sheet connection information:
>first sheet name
>
>second sheet name
>
>num of instances on second sheet

## Next Steps:
**Create input so that there are two dates inputted. Users can choose to either do +7 or +anything they want. Change sheetFinder code to include this change. Maybe break it up so that it runs in a different function or perhaps use recursion?**

In [7]:
from datetime import date, datetime

def leapYear(date_inp):
    year = int(date_inp[0:4])
    if (year % 4) == 0:
        if (year % 100) == 0:
            if (year % 400) == 0:
                #is leap year
                return True
            else:
                #not leap year
                return False
        else:
            #is
            return True
    else:
        #not
        return False

def regionFinder(state_inp):
    east_arr = ['Alabama', 'Florida', 'Georgia', 'North Carolina', 'South Carolina', 'Virginia', 'West Virginia', 'Kentucky', 'New Jersey', 'Tennessee', 'Connecticut', 'Delaware', 'District of Columbia', 'Maine', 'Maryland', 'Massachusetts', 'New Hampshire', 'New York', 'Rhode Island', 'Vermont']
    mt_arr = ['Colorado', 'Minnesota', 'Montana', 'Nebraska', 'New Mexico', 'North Dakota', 'South Dakota', 'Texas', 'Wyoming', 'Illinois', 'Indiana', 'Michigan', 'Ohio', 'Pennsylvania', 'Wisconsin']
    
    if state_inp in mt_arr:
        return 'Mountains-Midwest'
    elif state_inp in east_arr:
        return 'Southeast-East-Northeast'
    else:
        return 'West-Prairie-RiverValley'

def sheetFinder(state_inp, date_inp):
    region_str = regionFinder(state_inp) #region returned
    #dates arr could be problematic if naming conventions change
    dates = ['1-20-2021 to present','12-02-2020 to 1-01-2021','10-28-2020 to 12-01-2020','9-15-2020 to 10-27-2020','7-1-2020 to 9-14-2020','5-01-2020 to 6-30-2020','1-20-2020 to 4-30-2020']
    
    for i in range(dates.length):
        if i == 0:
            #current wksht - BASE CASE
            sheet_str = region_str + ' COVID-19 by County (TEAM ENTRY) ' + date[i]
        else:
            sheet_str = 'HISTORIC - ' + region_str + ' COVID-19 by County (TEAM ENTRY) ' + date[i]
        conn = gc.open(sheet_str)
        if state_inp == 'Missouri':
            worksheet = conn.get_worksheet(1)
        else:
            worksheet = conn.get_worksheet(0)
        frmt_date_inp = date.fromisoformat(date_inp)
        str_date_inp = str(frmt_date_inp.month) + "/" + str(frmt_date_inp.day) + "/" + str(frmt_date_inp.year)
        num_date_inp = int(frmt_date_inp.strftime("%j"))
        try:
            worksheet.find(str_date_inp)
            #once you've found the worksheet, check upper bound of 7-day span
            uppr_date = num_date_inp + 7
            ly = leapYear(date_inp)
            if ly == True & uppr_date > 366:
                diff = uppr_date - 366
            elif ly == False & uppr_date > 365:
                diff = uppr_date - 365
            else:
                ret_dict = {"sheetstr":sheet_str}
                return ret_dict
                break
            str_uppr_date = datetime.strptime(str(diff),'%j')
            new_form = str_uppr_date.strftime("%x")
            newyear = int(date_inp[0:4])+1
            skippy = new_form[:-2] + str(newyear)
            sheet2_str = 'HISTORIC - ' + region_str + ' COVID-19 by County (TEAM ENTRY) ' + date[i-1]
            ret_str = "endpoint: " + skippy + ", diff: " + diff
            ret_dict = {"sheet1str": sheet_str, "sheet2str": sheet2str, "details":ret_str}
            return ret_str
        except:
            i = i+1
            continue
            

In [8]:
import datetime

'''
ARGS:
state_inp: String
date_inp: String 'yyyy-mm-dd' must be in this format in order to allow <, > ops
'''

def countyThing(state_inp, county_inp, date_inp,worksheet):
    #do i need to list worksheet in inputs again?
    countyStr = county_inp + ', ' + state_inp
    #worksheet declared in sheetChoice function
    countyRow = worksheet.find(countryStr).row
    
    formatDate = date.fromisoformat(date_inp)
    strDate = formatDate.month + "/" + formatDate.day + "/"+ formatDate.year
    countyCol = worksheet.find(strDate).col
    

## Internal Data Analysis:
- Problematic Trends
- *DO* County Comparisons

Most Recent Update: **01/13/2021**

In [ ]:
'''
Args:
fip: int - state fip
mortality: bool
prob: bool

How to look at different columns:
tstpos: mortality_False, prob_False
mort: mortality_True, prob_False
pbpos: mortality_False, prob_True
pbmort: mortality_True, prob_True
'''

def problematicStateTrend(fip, mortality, prob):
#First let us find the row that totals are in, this will be done by finding the Alaska FIP:
    cell_fip = worksheet.find(fip) #AL: "04000US02"
    row_num_fip = cell_fip.row # Get row number = 4
    #col_num_AL = cell_fip.col

# Creating the column iterator to line up with tstpos, mort, pbpos, or pbmort
    if prob == False:
        if mortality == False:
            col_iterator = cell_fip.col + 3 #looking at tstpos
        else:
            col_iterator = cell_fip.col + 5 #looking at mort
    else:
        if mortality == False:
            col_iterator = cell_fip.col + 4 #looking at pbpos 
        else:
            col_iterator = cell_fip.col + 6 #looking at pbmort
        


#POTENTIAL ISSUE:
#130 was "random", columns with data < 130 (as of rn.)
#because we don't want to have to keep updating the "130" as data gets added, this should be workshopped.
# could do array.link -1 or +1 depending on where you start


#dstruc options:
# 1. LIST: each index has a long string of info
# 2. MULTIPLE LISTS: (dates, deltas (decreases), )
# 3. DICT: key: date, val: delta?
# 2d array


    dates_AL = [] #for now using this as DSTRUC.
    while(col_iterator < 130): #130 was "random", columns with data < 130 (as of rn.) because we don't want to have to keep updating the "130" as data gets added, this should be workshopped. 
        try:
            prev = int(worksheet.cell(row_num_fip, col_iterator).value)
            #print(prev)
            curr = int(worksheet.cell(row_num_fip, col_iterator+4).value)
            #print(curr<prev)
            if curr < prev:
                delta = prev - curr
                str_temp = 'Date: ' + worksheet.cell(1, col_iterator+4).value + ' Decrease by: ' + delta
                dates_AL.append(str_temp) #append the date & delta
            elif curr == prev & curr>300:
                temp_str = 'Static date: ' + worksheet.cell(1, col_iterator+4).value
                dates_AL.append(temp_str) #append the date
            col_iterator = col_iterator + 4
        except:
            #if it can't coerce an int should we add that as an error and add the date to the list... but would that be curr or prev, 
            #and how would we know which one turned up the error?
            col_iterator = col_iterator + 4 
            
        #finally:
            #return dates_AL
        #finally: will execute code regardless of try except
            
   
    return dates_AL    
 

## Execution

Most Recent Update: **01/13/2021**

In [ ]:
def main():
    fip = "04000US02"
    prob = False
    mortality = False
    print(problematicStateTrend(fip, mortality, prob))
    #print(dates_AL)

if __name__ == "__main__":
    main()